# Measuring impact of tolerances on Gurobi

In here we analyse the result of Gurobi runs using the same model but different parameter options. I used the `baseline` scenario and changed the `OptimalityTol` and `FeasibilityTol` values of Gurobi to:

* 1e-6 (default value)
* 1e-4
* 1e-2 (max value)

The hope being, that the result remains the same (remains close enough) but that solution time drops. Let's see whether that actually worked.
   

In [1]:
from pathlib import Path
import re

from scipy import stats

In [2]:
SOLVE_DURATION_PREFIX = "SOLVER   Solved in"
SOLUTION_PREFIX = "SOLVER   Optimal objective"


def _preprocess_log(path_to_log):
    path_to_log = Path(path_to_log)
    with path_to_log.open("r") as log_file:
        lines = log_file.readlines()
    return [line[22:] for line in lines]
    

def parse_solution_time(path_to_log):
    lines = _preprocess_log(path_to_log)
    duration_line = list(filter(lambda line: line.startswith(SOLVE_DURATION_PREFIX), lines))[0]
    return float(re.findall('\d+.?\d+', duration_line)[1])


def parse_iterations(path_to_log):
    lines = _preprocess_log(path_to_log)
    iterations_line = list(filter(lambda line: line.startswith(SOLVE_DURATION_PREFIX), lines))[0]
    return float(re.findall('\d+.?\d+', iterations_line)[0])


def parse_solution(path_to_log):
    lines = _preprocess_log(path_to_log)
    solution_line = list(filter(lambda line: line.startswith(SOLUTION_PREFIX), lines))[0]
    return float(re.findall('\d+.?\d+e?\+?\d+', solution_line)[0])

In [3]:
def analyse_runs(path_to_root_folder):
    durations = [parse_solution_time(path) for path in Path(path_to_root_folder).glob("*.log")]
    solutions = [parse_solution(path) for path in Path(path_to_root_folder).glob("*.log")]
    iterations = [parse_iterations(path) for path in Path(path_to_root_folder).glob("*.log")]
    
    mean, std = stats.norm.fit(solutions)
    print(f"Gurobi found the following optimal value on average: {mean} (std {std}).")
    mean, std = stats.norm.fit(iterations)
    print(f"Gurobi needed on average {mean:.0f} (std {std:.0f}) iterations to find the optimal value.")
    mean, std = stats.norm.fit(durations)
    print(f"Gurobi needed on average {mean:.0f}s (std {std:.0f}s) to find the optimal value.")

    
def diff(path_to_root_folder, path_to_root_folder_of_base):
    durations = [parse_solution_time(path) for path in Path(path_to_root_folder).glob("*.log")]
    solutions = [parse_solution(path) for path in Path(path_to_root_folder).glob("*.log")]
    iterations = [parse_iterations(path) for path in Path(path_to_root_folder).glob("*.log")]
    
    base_durations = [parse_solution_time(path) for path in Path(path_to_root_folder_of_base).glob("*.log")]
    base_solutions = [parse_solution(path) for path in Path(path_to_root_folder_of_base).glob("*.log")]
    base_iterations = [parse_iterations(path) for path in Path(path_to_root_folder_of_base).glob("*.log")]
    
    mean, std = stats.norm.fit(solutions)
    mean_base, std_base = stats.norm.fit(base_solutions)
    print(f"Relative diff of objective average to base: {mean / mean_base}")
    mean, std = stats.norm.fit(iterations)
    mean_base, std_base = stats.norm.fit(base_iterations)
    print(f"Relative diff of iterations average to base: {mean / mean_base}")
    mean, std = stats.norm.fit(durations)
    mean_base, std_base = stats.norm.fit(base_durations)
    print(f"Relative diff of durations average to base: {mean / mean_base}")
    

In [4]:
print("Tolerances e-6")
print("--------------")
analyse_runs("minus6/")
print()
print("Tolerances e-4")
print("--------------")
analyse_runs("minus4/")
diff("minus4/", "minus6/")
print()
print("Tolerances e-2")
print("--------------")
analyse_runs("minus2/")
diff("minus2/", "minus6/")
print()

Tolerances e-6
--------------
Gurobi found the following optimal value on average: 336375706000.0 (std 0.0).
Gurobi needed on average 2927011 (std 0) iterations to find the optimal value.
Gurobi needed on average 10794s (std 1146s) to find the optimal value.

Tolerances e-4
--------------
Gurobi found the following optimal value on average: 336375403000.0 (std 0.0).
Gurobi needed on average 2249170 (std 0) iterations to find the optimal value.
Gurobi needed on average 18399s (std 1805s) to find the optimal value.
Relative diff of objective average to base: 0.9999990992215115
Relative diff of iterations average to base: 0.7684187042686208
Relative diff of durations average to base: 1.7046015000200583

Tolerances e-2
--------------
Gurobi found the following optimal value on average: 336375292600.0 (std 0.0).
Gurobi needed on average 887148 (std 0) iterations to find the optimal value.
Gurobi needed on average 4347s (std 641s) to find the optimal value.
Relative diff of objective average

## Results

Interestingly, the runs with the same parameter set always lead to the same objective value and the same number of iterations. The found objective value can be called close enough for all chosen tolerance values. The largest tolerance needed only a third of the number of iterations and only 40% of the time of the default value set (e-6).

BUT, Barrier seems to be sensitive to the tolerances: the medium value of e-4 performed worse in terms of time than the more tight parameter set of e-6, despite needing about 25% less iterations. Potentially, this may stem from the fact that the model has numerical issues, see ranges below. It would be good to solve the numerical issues through scaling and then repeat this experiment.

```
[2019-03-21 19:53:24] SOLVER   Matrix range     [1e-05, 4e+05]
[2019-03-21 19:53:24] SOLVER   Objective range  [1e+00, 1e+00]
[2019-03-21 19:53:24] SOLVER   Bounds range     [5e+01, 3e+06]
[2019-03-21 19:53:24] SOLVER   RHS range        [7e+00, 3e+06]
```


## Conclusion

I can say that the different tolerance values did not impact the solution in a strong way. Thus, changing the tolerances in this model is safe. However, I cannot give any recommendation regarding solution times. Solve time _can_ be much faster with tolerances that are more loose, but they can as well be worse.